<a href="https://colab.research.google.com/github/SylvanusAdonu/hello-world/blob/master/SABR_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
# Install required packages
!pip install yfinance numpy pandas scipy matplotlib seaborn plotly dash ipywidgets -q

import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import norm
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# ==================== SABR MODEL WITH STOCHASTIC RATES ====================

class SABR_StochasticRates:
    """
    SABR model with stochastic interest rates (multi-factor)
    """
    def __init__(self, alpha=0.2, beta=0.5, rho=-0.3, nu=0.4,
                 kappa_r=0.1, theta_r=0.02, sigma_r=0.01, rho_sr=0.2):
        self.alpha = alpha  # Initial volatility
        self.beta = beta    # CEV parameter
        self.rho = rho      # Correlation between asset and vol
        self.nu = nu        # Vol of vol
        self.kappa_r = kappa_r  # Mean reversion speed for rates
        self.theta_r = theta_r  # Long-term mean rate
        self.sigma_r = sigma_r  # Vol of rates
        self.rho_sr = rho_sr    # Correlation between asset and rates

    def implied_vol(self, F, K, T):
        """SABR implied volatility"""
        # Handle T <= 0 to avoid errors in SABR calculations
        if T <= 0:
            T = 1 / (365 * 24) # Minimum 1 hour to expiry

        if F == K:
            term1 = self.alpha / (F ** (1 - self.beta))
            term2 = 1 + (((1 - self.beta)**2)/24 * self.alpha**2/(F**(2 - 2*self.beta)) +
                        self.rho*self.beta*self.nu*self.alpha/(4*F**(1 - self.beta)) +
                        (2 - 3*self.rho**2)*self.nu**2/24)*T
            return term1 * term2
        else:
            z = (self.nu/self.alpha) * (F*K)**((1-self.beta)/2) * np.log(F/K)
            # Handle potential division by zero or log of non-positive
            if abs(1 - self.rho) < 1e-10:
                return np.nan # Or raise an error, depending on desired behavior
            x_z = np.log((np.sqrt(1 - 2*self.rho*z + z**2) + z - self.rho)/(1 - self.rho))
            term1 = self.alpha / ((F*K)**((1-self.beta)/2) * (1 + ((1-self.beta)**2/24)*(np.log(F/K)**2) +
                    ((1-self.beta)**4/1920)*(np.log(F/K)**4)))
            if abs(z) < 1e-10 or abs(x_z) < 1e-10:
                return term1
            term2 = z / x_z
            term3 = 1 + (((1-self.beta)**2/24)*(self.alpha**2/((F*K)**(1-self.beta))) +
                    (self.rho*self.beta*self.nu*self.alpha)/(4*((F*K)**((1-self.beta)/2))) +
                    ((2-3*self.rho**2)*self.nu**2)/24) * T
            return term1 * term2 * term3

    def calibrate(self, strikes, vols, F, T):
        """Calibrate SABR parameters to market data"""
        # Ensure T is always positive for calibration
        if T <= 0:
            T = 1 / (365 * 24) # Minimum 1 hour to expiry

        def objective(params):
            alpha, beta, rho, nu = params
            self.alpha, self.beta, self.rho, self.nu = alpha, beta, rho, nu
            mse = 0
            for K, vol in zip(strikes, vols):
                sabr_vol = self.implied_vol(F, K, T)
                if not np.isnan(sabr_vol): # Only include valid SABR vols in MSE
                    mse += (sabr_vol - vol)**2
                else:
                    mse += (vol)**2 # Penalize NaN results
            return mse

        # Initial guess and bounds
        initial_guess = [0.2, 0.5, -0.3, 0.4]
        bounds = [(0.01, 1), (0, 1), (-0.99, 0.99), (0.01, 2)]
        result = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
        self.alpha, self.beta, self.rho, self.nu = result.x
        return result.x

# ==================== HESTON WITH JUMPS (BATES MODEL) ====================

class BatesModel:
    """
    Heston model with jumps (Bates model)
    """
    def __init__(self, v0=0.04, kappa=1.5, theta=0.04, sigma=0.3, rho=-0.7,
                 lambd=0.2, mu_j=-0.1, delta_j=0.1, r=0.02):
        self.v0 = v0      # Initial variance
        self.kappa = kappa  # Mean reversion speed
        self.theta = theta  # Long-term variance
        self.sigma = sigma  # Vol of vol
        self.rho = rho      # Correlation
        self.lambd = lambd  # Jump intensity
        self.mu_j = mu_j    # Mean jump size
        self.delta_j = delta_j  # Jump size volatility
        self.r = r          # Risk-free rate

    def characteristic_function(self, phi, S0, K, T, opt_type='call'):
        """Characteristic function for Bates model"""
        # Ensure T is always positive for Bates model
        if T <= 0:
            T = 1 / (365 * 24) # Minimum 1 hour to expiry

        # Heston part
        D = np.sqrt((self.kappa - 1j*self.rho*self.sigma*phi)**2 + self.sigma**2*(1j*phi + phi**2))
        G = (self.kappa - 1j*self.rho*self.sigma*phi - D) / (self.kappa - 1j*self.rho*self.sigma*phi + D)
        C = self.kappa * (self.theta/self.sigma**2) * ((self.kappa - 1j*self.rho*self.sigma*phi - D)*T -
            2*np.log((1 - G*np.exp(-D*T))/(1 - G)))
        D_term = (self.kappa - 1j*self.rho*self.sigma*phi - D)/(self.sigma**2) * ((1 - np.exp(-D*T))/(1 - G*np.exp(-D*T)))

        # Jump part
        jump_exp = self.lambd * T * (np.exp(1j*phi*self.mu_j - 0.5*self.delta_j**2*phi**2) - 1)

        return np.exp(C + D_term*self.v0 + 1j*phi*np.log(S0) + jump_exp)

    def price(self, S0, K, T):
        """Price option using FFT (simplified implementation)"""
        # Ensure T is always positive for Bates model
        if T <= 0:
            T = 1 / (365 * 24) # Minimum 1 hour to expiry

        # Simplified pricing for demonstration
        # In production, use proper FFT implementation
        k = np.log(K/S0)
        integral = 0
        for phi in np.linspace(0.1, 100, 1000):
            # Ensure the denominator is not zero
            denominator = (1j*phi*(1j*phi + 1))
            if abs(denominator) < 1e-10:
                continue
            psi = self.characteristic_function(phi - 1j, S0, K, T) / denominator
            integral += np.real(np.exp(-1j*phi*k) * psi) * 0.1

        return S0 - np.sqrt(S0*K)/np.pi * integral

# ==================== GEX CALCULATION FUNCTIONS ====================

def black_scholes_greeks(S, K, T, r, sigma, option_type='call'):
    """Calculate Black-Scholes greeks"""
    # Ensure T is always positive for Black-Scholes
    if T <= 0:
        T = 1 / (365 * 24) # Minimum 1 hour to expiry

    # Avoid division by zero or log of non-positive
    if sigma <= 0 or T <= 0:
        return {'delta': np.nan, 'gamma': np.nan, 'vega': np.nan, 'theta': np.nan,
                'vanna': np.nan, 'charm': np.nan}

    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if option_type == 'call':
        gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)
        theta = (-S * norm.pdf(d1) * sigma / (2*np.sqrt(T)) - r * K * np.exp(-r*T) * norm.cdf(d2))
        delta = norm.cdf(d1)
        vanna = -norm.pdf(d1) * d2 / sigma
        charm = -norm.pdf(d1) * (2*r*T - d2*sigma*np.sqrt(T)) / (2*T*sigma*np.sqrt(T))
    else:  # put
        gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)
        theta = (-S * norm.pdf(d1) * sigma / (2*np.sqrt(T)) + r * K * np.exp(-r*T) * norm.cdf(-d2))
        delta = norm.cdf(d1) - 1
        vanna = -norm.pdf(d1) * d2 / sigma
        charm = -norm.pdf(d1) * (2*r*T - d2*sigma*np.sqrt(T)) / (2*T*sigma*np.sqrt(T))

    return {'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta,
            'vanna': vanna, 'charm': charm}

def calculate_gex(strikes, volumes, open_interest, S, T, r, ivs, option_types):
    """Calculate Gamma Exposure for each strike"""
    gex_values = []
    greeks_data = []

    for K, volume, oi, iv, opt_type in zip(strikes, volumes, open_interest, ivs, option_types):
        greeks = black_scholes_greeks(S, K, T, r, iv, opt_type)

        # Gamma exposure = gamma * spot² * 0.01 * volume
        # Ensure gamma is not NaN before calculation
        if not np.isnan(greeks['gamma']):
            gex = greeks['gamma'] * (S**2) * 0.01 * volume * 100  # *100 for contract multiplier
        else:
            gex = np.nan # If gamma is NaN, GEX is NaN

        gex_values.append(gex)
        greeks_data.append({
            'strike': K,
            'delta': greeks['delta'],
            'gamma': greeks['gamma'],
            'vega': greeks['vega'],
            'vanna': greeks['vanna'],
            'charm': greeks['charm'],
            'gex': gex,
            'volume': volume,
            'open_interest': oi,
            'iv': iv,
            'type': opt_type
        })

    return gex_values, greeks_data

# ==================== DATA FETCHING AND PROCESSING ====================

def get_spx_options_data(target_expiry_date=None):
    """Fetch SPX options data for 0DTE or a specified date"""
    spx = yf.Ticker("^SPX")

    # Get current price
    try:
        current_price = spx.history(period='1d')['Close'].iloc[-1]
    except IndexError:
        print("Warning: Could not fetch SPX current price. Market might be closed or data unavailable. Using a placeholder.")
        current_price = 4500 # Placeholder price

    # Get today's date
    today = datetime.now()

    if target_expiry_date:
        # Use the specified expiry date
        try:
            exp_datetime = datetime.strptime(target_expiry_date, '%Y-%m-%d')
            closest_expiry = target_expiry_date
        except ValueError:
            print(f"Warning: Invalid date format for target_expiry_date: {target_expiry_date}. Falling back to closest expiry.")
            target_expiry_date = None # Fallback

    if not target_expiry_date:
        # Find the closest expiry that is today or in the future
        expiries = spx.options
        closest_expiry = None
        min_time_diff = timedelta(days=9999) # Initialize with a large difference

        for exp in expiries:
            exp_date_obj = datetime.strptime(exp, '%Y-%m-%d')
            time_diff = exp_date_obj - today

            # We are looking for expiries that are today or in the future
            # and are the closest to 'today' (i.e., minimal positive time_diff)K
            if time_diff.total_seconds() >= 0 and time_diff < min_time_diff:
                closest_expiry = exp
                min_time_diff = time_diff

        if not closest_expiry:
            print("Error: No expiry dates available for SPX at all. Returning empty data.")
            return pd.DataFrame(), current_price, 1 / (365 * 24), None # Smallest T, empty options

        exp_datetime = datetime.strptime(closest_expiry, '%Y-%m-%d')

    # Calculate time to expiration in years
    T = ((exp_datetime - today).total_seconds() / (365 * 24 * 3600))

    # Ensure T is always positive, even if it's very small or negative (e.g., past expiry)
    if T <= 0:
        # Set a minimum non-zero time to expiry (e.g., 1 hour)
        T = 1 / (365 * 24)

    # Get options chain
    opt_chain = spx.option_chain(closest_expiry)

    # Combine calls and puts
    calls = opt_chain.calls
    puts = opt_chain.puts

    # Add option type column
    calls['type'] = 'call'
    puts['type'] = 'put'

    # Combine and filter near ATM options
    all_options = pd.concat([calls, puts])

    # Filter for reasonable strikes
    all_options = all_options[
        (all_options['strike'] > current_price * 0.8) &
        (all_options['strike'] < current_price * 1.2)
    ]

    return all_options, current_price, T, closest_expiry

# ==================== MAIN ANALYSIS DASHBOARD ====================

def create_gex_dashboard(target_expiry_date=None):
    """Create comprehensive GEX analysis dashboard"""

    print("="*80)
    print("SPX 0DTE GAMMA EXPOSURE FLOW ANALYSIS")
    print("="*80)

    # Fetch data
    print("\n📊 Fetching SPX options data...")
    options_data, S, T, expiry = get_spx_options_data(target_expiry_date=target_expiry_date)

    if options_data.empty:
        print("No options data available after filtering! Cannot create dashboard.")
        return pd.DataFrame(), None, None # Return empty dataframe, None for SABR model, and None for S

    print(f"Current SPX: ${S:.2f}")
    print(f"Expiry: {expiry} (T={T:.4f} years)")
    print(f"Total options: {len(options_data)}")

    # Assume risk-free rate (can be fetched from Treasury yields)
    r = 0.045  # Current ~4.5%

    # Calculate GEX
    print("\n🧮 Calculating Gamma Exposure...")
    gex_values, greeks_data = calculate_gex(
        options_data['strike'].values,
        options_data['volume'].fillna(0).values,
        options_data['openInterest'].fillna(0).values,
        S, T, r,
        options_data['impliedVolatility'].fillna(0.2).values,
        options_data['type'].values
    )

    greeks_df = pd.DataFrame(greeks_data)

    # Drop rows where GEX or other key metrics are NaN, as they can't be plotted meaningfully
    greeks_df.dropna(subset=['gex', 'gamma', 'delta', 'iv'], inplace=True)

    if greeks_df.empty:
        print("No valid GEX data after dropping NaNs. Cannot create dashboard.")
        return pd.DataFrame(), None, None

    # Separate calls and puts
    calls_df = greeks_df[greeks_df['type'] == 'call']
    puts_df = greeks_df[greeks_df['type'] == 'put']

    # ==================== PLOTTING ====================

    fig = make_subplots(
        rows=4, cols=3,
        subplot_titles=('Dealer Gamma Exposure by Strike',
                       'Call vs Put GEX Breakdown',
                       'Open Interest Distribution',
                       'GEX by Volume',
                       'Implied Volatility Smile (SABR)',
                       'Skew Analysis by Delta',
                       'Vanna Exposure by Strike',
                       'Charm Exposure by Strike',
                       'GEX Weighted IV Analysis',
                       'Total GEX Profile',
                       'Delta Distribution',
                       'Gamma Distribution'),
        vertical_spacing=0.08,
        horizontal_spacing=0.1,
        specs=[[{'type': 'scatter'}, {'type': 'bar'}, {'type': 'bar'}],
               [{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'scatter'}]]
    )

    # 1. Dealer Gamma Exposure by Strike
    calls_gex = calls_df.groupby('strike')['gex'].sum()
    puts_gex = puts_df.groupby('strike')['gex'].sum()
    total_gex = greeks_df.groupby('strike')['gex'].sum()

    fig.add_trace(
        go.Scatter(x=total_gex.index, y=total_gex.values,
                  mode='lines+markers', name='Total GEX',
                  line=dict(color='blue', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=calls_gex.index, y=calls_gex.values,
                  mode='lines', name='Call GEX',
                  line=dict(color='green', dash='dash')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=puts_gex.index, y=puts_gex.values,
                  mode='lines', name='Put GEX',
                  line=dict(color='red', dash='dash')),
        row=1, col=1
    )
    fig.add_vline(x=S, line=dict(color="gold", width=2, dash="dot"),
                  row=1, col=1, annotation_text="Spot")

    # 2. Call vs Put GEX Breakdown
    total_call_gex = calls_df['gex'].sum()
    total_put_gex = puts_df['gex'].sum()

    fig.add_trace(
        go.Bar(x=['Calls', 'Puts'],
               y=[total_call_gex, total_put_gex],
               marker_color=['green', 'red'],
               text=[f'${total_call_gex/1e9:.2f}B', f'${total_put_gex/1e9:.2f}B'],
               textposition='auto'),
        row=1, col=2
    )

    # 3. Open Interest Distribution
    fig.add_trace(
        go.Bar(x=greeks_df['strike'], y=greeks_df['open_interest'],
               marker_color='purple', name='Open Interest'),
        row=1, col=3
    )
    fig.add_vline(x=S, line=dict(color="gold", width=2, dash="dot"),
                  row=1, col=3)

    # 4. GEX by Volume
    max_abs_gex_volume = np.abs(greeks_df['gex']).max()
    if pd.isna(max_abs_gex_volume) or max_abs_gex_volume == 0:
        marker_sizes_volume = 5 # Default size if no meaningful GEX values
    else:
        marker_sizes_volume = np.abs(greeks_df['gex'])/max_abs_gex_volume*50

    fig.add_trace(
        go.Scatter(x=greeks_df['strike'], y=greeks_df['volume'],
                   mode='markers',
                   marker=dict(
                       size=marker_sizes_volume,
                       color=greeks_df['gex'],
                       colorscale='RdBu',
                       showscale=True,
                       colorbar=dict(title="GEX")
                   ),
                   name='GEX by Volume'),
        row=2, col=1
    )
    fig.add_vline(x=S, line=dict(color="gold", width=2, dash="dot"),
                  row=2, col=1)

    # 5. Implied Volatility Smile (SABR)
    sabr_model = None
    # Fit SABR model
    if len(greeks_df) > 0 and T > 0:
        # Ensure there are enough data points for calibration
        sample_size = min(len(greeks_df), 50)
        strikes_sample = greeks_df['strike'].values[:sample_size]
        ivs_sample = greeks_df['iv'].values[:sample_size]

        if len(strikes_sample) > 0 and not np.all(np.isnan(ivs_sample)):
            sabr_model = SABR_StochasticRates()
            try:
                sabr_model.calibrate(strikes_sample, ivs_sample, S, T)

                # Generate SABR curve
                sabr_strikes = np.linspace(greeks_df['strike'].min(), greeks_df['strike'].max(), 100)
                sabr_vols = [sabr_model.implied_vol(S, K, T) for K in sabr_strikes]

                fig.add_trace(
                    go.Scatter(x=sabr_strikes, y=sabr_vols,
                               mode='lines', name='SABR Fit',
                               line=dict(color='red', width=2)),
                    row=2, col=2
                )
            except Exception as e:
                print(f"Warning: SABR calibration failed: {e}")
        else:
            print("Warning: Not enough valid IV data for SABR calibration.")
    else:
        print("Warning: No options data or invalid T for SABR calibration.")

    fig.add_trace(
        go.Scatter(x=greeks_df['strike'], y=greeks_df['iv'],
                   mode='markers', name='Market IV',
                   marker=dict(color='blue', size=8)),
        row=2, col=2
    )
    fig.add_vline(x=S, line=dict(color="gold", width=2, dash="dot"),
                  row=2, col=2)

    # 6. Skew Analysis by Delta
    # Calculate delta buckets (ensuring consistency with summary stats)
    greeks_df['delta_abs'] = np.abs(greeks_df['delta'])

    delta_buckets_plot = {
        '15Δ': greeks_df[np.abs(greeks_df['delta_abs'] - 0.15) < 0.05],
        '25Δ': greeks_df[np.abs(greeks_df['delta_abs'] - 0.25) < 0.05],
        '50Δ': greeks_df[np.abs(greeks_df['delta_abs'] - 0.50) < 0.05]
    }

    delta_categories = []
    call_ivs = []
    put_ivs = []
    skews = []

    for name, df_bucket in delta_buckets_plot.items():
        delta_categories.append(name)

        call_iv = df_bucket[df_bucket['type'] == 'call']['iv'].mean()
        put_iv = df_bucket[df_bucket['type'] == 'put']['iv'].mean()

        # Handle NaN values for mean if a bucket is empty
        call_iv = call_iv if not pd.isna(call_iv) else 0.0
        put_iv = put_iv if not pd.isna(put_iv) else 0.0

        skew = put_iv - call_iv

        call_ivs.append(call_iv)
        put_ivs.append(put_iv)
        skews.append(skew)

    fig.add_trace(
        go.Bar(x=delta_categories, y=call_ivs, name='Call IV', marker_color='green'),
        row=2, col=3
    )
    fig.add_trace(
        go.Bar(x=delta_categories, y=put_ivs, name='Put IV', marker_color='red'),
        row=2, col=3
    )
    fig.add_trace(
        go.Bar(x=delta_categories, y=skews, name='Skew (Put - Call)', marker_color='blue'),
        row=2, col=3
    )

    fig.update_layout(barmode='group') # Apply grouped bar mode globally for this subplot

    # 7. Vanna Exposure by Strike
    fig.add_trace(
        go.Scatter(x=greeks_df['strike'], y=greeks_df['vanna'],
                   mode='lines+markers',
                   line=dict(color='purple', width=2),
                   name='Vanna'),
        row=3, col=1
    )
    fig.add_vline(x=S, line=dict(color="gold", width=2, dash="dot"),
                  row=3, col=1)

    # 8. Charm Exposure by Strike
    fig.add_trace(
        go.Scatter(x=greeks_df['strike'], y=greeks_df['charm'],
                   mode='lines+markers',
                   line=dict(color='brown', width=2),
                   name='Charm'),
        row=3, col=2
    )
    fig.add_vline(x=S, line=dict(color="gold", width=2, dash="dot"),
                  row=3, col=2)

    # 9. GEX Weighted IV Analysis
    max_abs_gex_iv = np.abs(greeks_df['gex']).max()
    if pd.isna(max_abs_gex_iv) or max_abs_gex_iv == 0:
        marker_sizes_iv = 5 # Default size if no meaningful GEX values
    else:
        marker_sizes_iv = np.abs(greeks_df['gex'])/max_abs_gex_iv*50

    fig.add_trace(
        go.Scatter(x=greeks_df['strike'], y=greeks_df['iv'],
                   mode='markers',
                   marker=dict(
                       size=marker_sizes_iv,
                       color=greeks_df['gex'],
                       colorscale='Viridis',
                       showscale=True,
                       colorbar=dict(title="GEX")
                   ),
                   name='GEX Weighted IV'),
        row=3, col=3
    )
    fig.add_vline(x=S, line=dict(color="gold", width=2, dash="dot"),
                  row=3, col=3)

    # 10. Total GEX Profile
    total_gex_sorted = total_gex.sort_index()
    fig.add_trace(
        go.Scatter(x=total_gex_sorted.index, y=total_gex_sorted.values.cumsum(),
                   mode='lines',
                   line=dict(color='blue', width=3),
                   name='Cumulative GEX'),
        row=4, col=1
    )
    fig.add_vline(x=S, line=dict(color="gold", width=2, dash="dot"),
                  row=4, col=1)

    # 11. Delta Distribution
    fig.add_trace(
        go.Histogram(x=greeks_df['delta'], nbinsx=50,
                     marker_color='lightblue',
                     name='Delta Distribution'),
        row=4, col=2
    )

    # 12. Gamma Distribution
    fig.add_trace(
        go.Histogram(x=greeks_df['gamma'], nbinsx=50,
                     marker_color='lightgreen',
                     name='Gamma Distribution'),
        row=4, col=3
    )

    # Update layout
    fig.update_layout(
        height=1600,
        showlegend=True,
        title_text=f"SPX 0DTE GEX Dashboard | Spot: ${S:.2f} | Expiry: {expiry}",
        title_font_size=20
    )

    # Update axes labels
    for i in range(1, 5):
        for j in range(1, 4):
            fig.update_xaxes(title_text="Strike", row=i, col=j)
            if j == 1:
                fig.update_yaxes(title_text="Gamma Exposure", row=i, col=j)
    # Specific updates for the skew analysis subplot
    fig.update_xaxes(title_text="Delta", row=2, col=3)
    fig.update_yaxes(title_text="Implied Volatility / Skew", row=2, col=3)

    fig.show()

    # ==================== SUMMARY STATISTICS ====================

    print("\n" + "="*80)
    print("SUMMARY STATISTICS")
    print("="*80)

    # Calculate key metrics
    total_gex_value = greeks_df['gex'].sum()
    net_gamma_exposure = total_gex_value

    # Find gamma flip zones
    gex_by_strike = greeks_df.groupby('strike')['gex'].sum()
    positive_gex = gex_by_strike[gex_by_strike > 0]
    negative_gex = gex_by_strike[gex_by_strike < 0]

    if len(positive_gex) > 0:
        max_positive_strike = positive_gex.idxmax()
        max_positive_value = positive_gex.max()
    else:
        max_positive_strike = None
        max_positive_value = 0

    if len(negative_gex) > 0:
        max_negative_strike = negative_gex.idxmin()
        max_negative_value = negative_gex.min()
    else:
        max_negative_strike = None
        max_negative_value = 0

    # Calculate vanna and charm exposure
    total_vanna = greeks_df['vanna'].sum()
    total_charm = greeks_df['charm'].sum()

    # Calculate skew
    atm_iv = greeks_df[np.abs(greeks_df['strike'] - S) < S*0.01]['iv'].mean()
    # Using the same delta_buckets_plot for consistency, extract relevant parts
    skew_25d_calc_df = delta_buckets_plot['25Δ']
    skew_25d_call_iv = skew_25d_calc_df[skew_25d_calc_df['type'] == 'call']['iv'].mean()
    skew_25d_put_iv = skew_25d_calc_df[skew_25d_calc_df['type'] == 'put']['iv'].mean()
    skew_25d = (skew_25d_put_iv - skew_25d_call_iv) if not (pd.isna(skew_25d_put_iv) or pd.isna(skew_25d_call_iv)) else 0

    skew_15d_calc_df = delta_buckets_plot['15Δ']
    skew_15d_call_iv = skew_15d_calc_df[skew_15d_calc_df['type'] == 'call']['iv'].mean()
    skew_15d_put_iv = skew_15d_calc_df[skew_15d_calc_df['type'] == 'put']['iv'].mean()
    skew_15d = (skew_15d_put_iv - skew_15d_call_iv) if not (pd.isna(skew_15d_put_iv) or pd.isna(skew_15d_call_iv)) else 0

    print(f"\n📈 Total Gamma Exposure: ${total_gex_value/1e9:.2f} Billion")
    print(f"📉 Net Gamma Position: {'Positive' if net_gamma_exposure > 0 else 'Negative'}")
    # Handle None values for strikes in print statements
    print(f"🎯 Maximum Positive GEX: ${max_positive_value/1e6:.2f}M at {f'${max_positive_strike:.0f}' if max_positive_strike is not None else 'N/A'}")
    print(f"🎯 Maximum Negative GEX: ${max_negative_value/1e6:.2f}M at {f'${max_negative_strike:.0f}' if max_negative_strike is not None else 'N/A'}")
    print(f"🔄 Total Vanna Exposure: {total_vanna:.2e}")
    print(f"⏳ Total Charm Exposure: {total_charm:.2e}")
    print(f"📊 25Δ Skew (Put IV - Call IV): {skew_25d:.3f}")
    print(f"📊 15Δ Skew (Put IV - Call IV): {skew_15d:.3f}")
    print(f"🎯 ATM IV: {atm_iv:.3f}")

    # Market implications
    print("\n" + "="*80)
    print("MARKET IMPLICATIONS")
    print("="*80)

    if net_gamma_exposure > 0:
        print("🔵 POSITIVE GAMMA DOMINANT:")
        print("   • Market makers are net long gamma")
        print("   • Hedging flows are stabilizing")
        print("   • Expect reduced volatility near current levels")
    else:
        print("🔴 NEGATIVE GAMMA DOMINANT:")
        print("   • Market makers are net short gamma")
        print("   • Hedging flows amplify price moves")
        print("   • Risk of accelerated moves if spot breaks key levels")

    if total_vanna > 0:
        print("\n📈 POSITIVE VANNA:")
        print("   • IV increases as spot rises")
        print("   • Calls gain more value on upside moves")
    else:
        print("\n📉 NEGATIVE VANNA:")
        print("   • IV increases as spot falls")
        print("   • Puts gain more value on downside moves")

    # Key levels to watch
    print("\n" + "="*80)
    print("KEY LEVELS TO WATCH")
    print("="*80)

    # Find largest gamma concentrations
    top_5_gex = gex_by_strike.nlargest(5)
    bottom_5_gex = gex_by_strike.nsmallest(5)

    print("\nTop 5 Gamma Resistant Levels (Support):")
    for strike, gex in top_5_gex.items():
        print(f"  • ${strike:.0f}: ${gex/1e6:.1f}M GEX")

    print("\nTop 5 Gamma Vacuum Levels (Resistance):")
    for strike, gex in bottom_5_gex.items():
        print(f"  • ${strike:.0f}: ${gex/1e6:.1f}M GEX")

    # Calculate Gamma Wall
    gamma_wall = gex_by_strike.idxmax() if len(gex_by_strike) > 0 else S
    print(f"\n🏰  Gamma Wall: ${gamma_wall:.0f}")

    return greeks_df, sabr_model, S

# ==================== RUN THE ANALYSIS ====================

if __name__ == "__main__":
    # To specify an expiry date, uncomment the line below and provide a date in 'YYYY-MM-DD' format.
    # For example: specific_expiry = '2024-06-21'
    specific_expiry = None # Set to None to automatically find the closest expiry

    # Create the dashboard
    greeks_df, sabr_model, S = create_gex_dashboard(target_expiry_date=specific_expiry)

    if not greeks_df.empty:
        # Additional advanced analysis
        print("\n" + "="*80)
        print("ADVANCED ANALYSIS")
        print("="*80)

        # Bates model analysis
        print("\n🎯 Bates Model (Heston with Jumps) Analysis:")
        bates = BatesModel()

        # Calculate jump-adjusted prices for sample strikes
        sample_strikes = [greeks_df['strike'].min(),
                         greeks_df['strike'].median(),
                         greeks_df['strike'].max()]

        print("\nJump-Adjusted Option Premiums (Bates Model):")
        # Approximate spot and T for Bates model, ensuring T is positive
        S_bates = greeks_df['strike'].mean()
        T_bates = greeks_df['T'].iloc[0] if 'T' in greeks_df.columns and not greeks_df['T'].empty else 1/365 # Use actual T or a default
        if T_bates <= 0: T_bates = 1/365

        for K in sample_strikes[:3]:  # First 3 strikes
            price = bates.price(S_bates, K, T_bates)
            print(f"  Strike ${K:.0f}: ${price:.2f}")

        # Calculate Gamma Tilt
        print("\n∠ Gamma Tilt Analysis:")
        atm_gamma = greeks_df[np.abs(greeks_df['strike'] - S) < 5]['gamma'].mean()
        otm_gamma = greeks_df[greeks_df['strike'] > S*1.05]['gamma'].mean()
        itm_gamma = greeks_df[greeks_df['strike'] < S*0.95]['gamma'].mean()

        print(f"  ATM Gamma: {atm_gamma:.6f}")
        print(f"  OTM Gamma: {otm_gamma:.6f}")
        print(f"  ITM Gamma: {itm_gamma:.6f}")
        # Handle division by zero for Gamma Tilt
        if itm_gamma != 0:
            print(f"  Gamma Tilt (OTM/ITM): {otm_gamma/itm_gamma:.3f}")
        else:
            print("  Gamma Tilt (OTM/ITM): Undefined (ITM Gamma is zero)")

        # Exposure by moneyness
        print("\n💰 Exposure by Moneyness:")
        for moneyness in ['ITM', 'ATM', 'OTM']:
            if moneyness == 'ITM':
                df = greeks_df[greeks_df['strike'] < S*0.98]
            elif moneyness == 'ATM':
                df = greeks_df[(greeks_df['strike'] >= S*0.98) & (greeks_df['strike'] <= S*1.02)]
            else:
                df = greeks_df[greeks_df['strike'] > S*1.02]

            gex_sum = df['gex'].sum()
            print(f"  {moneyness}: ${gex_sum/1e6:.1f}M GEX")
    else:
        print("Advanced analysis skipped due to no valid options data.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00
SPX 0DTE GAMMA EXPOSURE FLOW ANALYSIS

📊 Fetching SPX options data...
Current SPX: $6978.33
Expiry: 2026-01-14 (T=0.0011 years)
Total options: 299

🧮 Calculating Gamma Exposure...



SUMMARY STATISTICS

📈 Total Gamma Exposure: $2.31 Billion
📉 Net Gamma Position: Positive
🎯 Maximum Positive GEX: $386.59M at $6975
🎯 Maximum Negative GEX: $0.00M at N/A
🔄 Total Vanna Exposure: -1.79e+01
⏳ Total Charm Exposure: 2.05e+03
📊 25Δ Skew (Put IV - Call IV): -0.018
📊 15Δ Skew (Put IV - Call IV): -0.011
🎯 ATM IV: 0.093

MARKET IMPLICATIONS
🔵 POSITIVE GAMMA DOMINANT:
   • Market makers are net long gamma
   • Hedging flows are stabilizing
   • Expect reduced volatility near current levels

📉 NEGATIVE VANNA:
   • IV increases as spot falls
   • Puts gain more value on downside moves

KEY LEVELS TO WATCH

Top 5 Gamma Resistant Levels (Support):
  • $6975: $386.6M GEX
  • $6990: $373.7M GEX
  • $6980: $330.2M GEX
  • $7000: $250.8M GEX
  • $6985: $245.1M GEX

Top 5 Gamma Vacuum Levels (Resistance):
  • $6370: $0.0M GEX
  • $6380: $0.0M GEX
  • $6390: $0.0M GEX
  • $6410: $0.0M GEX
  • $6420: $0.0M GEX

🏰  Gamma Wall: $6975

ADVANCED ANALYSIS

🎯 Bates Model (Heston with Jumps) Analy